# ECON 490: Good Regression Practice (14)

## Prerequisites 

1. Importing data into Stata.
2. Creating new varables using `generate` and `replace`.
3. Identifying percentiles in data using `summarize` and `return list`.
4. Running OLS regressions.

## Learning Outcomes

1. Identify and correct for outliers by trimming or winsorizing the dependent variable. 
2. Identify and correct for the problem of multicollinearity.
3. Identify and correct for the problem of heteroskedasticity. 
4. Identify and correct for the problem of non-linearity. 

## 14.1 Dealing with Outliers 

Imagine that we have constructed a dependent variable which contains the earnings growth of individual workers and we see that some worker's earnings increased by more than 400%. We might wonder if this massive change is just a coding error made by the statisticians that produced the data set. Even without that type of error, though, we might worry that the earnings growth of a small number of observations are driving the results of our analysis. If this is the case, we will produce an inaccurate analysis based on results that are not associated with the majority of our observations. 

The standard practice in these cases is to either winsorize or trim the subset of observations that are used in that regression. Both practices remove the outlier values in the dependent variable to allow us to produce a more accurate empirical analysis. In this section, we will look at both approaches.

<div class="alert alert-block alert-warning">
    
<b>Warning:</b> You should only consider fixing outliers when there is a clear reason to address this issue. Do not apply the tools below if the summary statistics in your data make sense to you in terms of abnormal values. For example, outliers might be a sign that your dependent and explanatory variables have a non-linear relationship. If that is the case, you will want to consider including an interaction term that addresses that non-linearity. 
    
</div>

#### 14.1.1 Winsorizing a dependent variable

Winsorizing is the process of limiting extreme values in the dependent variable to reduce the effect of (possibly erroneous) outliers. It consists of replacing values below the $a$ percentile by that percentile value, and values above the $b$ percentile by that percentile. Consider the following example using our fake data set:

In [21]:
clear all


use fake_data, clear 

In [22]:
su earnings, d


                          Earnings
-------------------------------------------------------------
      Percentiles      Smallest
 1%     2830.869       36.19316
 5%     6639.007        46.1867
10%     10220.93       51.08368       Obs             138,138
25%     20562.63       57.63494       Sum of wgt.     138,138

50%     43783.01                      Mean           84136.44
                        Largest       Std. dev.      252801.7
75%     92378.23       1.10e+07
90%     183237.5       1.38e+07       Variance       6.39e+10
95%     277388.4       3.80e+07       Skewness        143.227
99%     607200.1       6.36e+07       Kurtosis       32584.05


From the summary statistics above, we can see that that the income earned by the individual at the 1st percentile is 2,037.23 and that the lowest earner in the data set earned 8.88.  

We can also see that income earned by the individual at the 99th percentile is only 540,525.90 and that the highest earner in the data earned over 190,000,000!

This table suggests to us that there are large outliers in our dependent variable.

We want to get rid of these outliers by winsorizing our data set. What that means is replacing the earnings of all observations below the 1st percentile by exactly the earnings of the individual at the 1st percentile, and replacing the earnings of all observations above the 99th percentile by exactly the earnings of the individual at the 99th percentile. 

Recall that we can see how Stata stored the information in the previously run `summarize` command by using the command `return list`.

In [23]:
return list


scalars:
                r(p99) =  607200.125
                r(p95) =  277388.40625
                r(p90) =  183237.453125
                r(p75) =  92378.234375
                r(p50) =  43783.0078125
                r(p25) =  20562.630859375
                r(p10) =  10220.9267578125
                 r(p5) =  6639.00732421875
                 r(p1) =  2830.868896484375
                r(max) =  63573580
                r(min) =  36.19315719604492
                r(sum) =  11622440098.6643
           r(kurtosis) =  32584.05136018418
           r(skewness) =  143.2270435630384
                 r(sd) =  252801.7082990456
                r(Var) =  63908703718.91576
               r(mean) =  84136.44398112247
              r(sum_w) =  138138
                  r(N) =  138138


To winsorize this data, we do the following 3 step process:

1. We create a new variable called *earnings_winsor* which is identical to our _earnings_ variable (`gen earnings_winsor = earnings`). We choose to store the winsorized version of the dependent variable in a different variable so that we don't overwrite the original data set. 
2. If earnings are smaller than the 1st percentile, we replace the values of *earnings_winsor* with the earnings of the individual at the 1st percentile (stored in Stata in r(p1)).
3. If earnings are larger than the 1st percentile, we replace the values of *earnings_winsor* with the earnings of the individual at the 99th percentile (stored in Stata in r(p99).

You can run these commands yourself below:

In [24]:
gen earnings_winsor = earnings
replace earnings_winsor = r(p1) if earnings_winsor<r(p1)
replace earnings_winsor = r(p99) if earnings_winsor>r(p99)



(1,381 real changes made)

(1,381 real changes made)


Let's take a look at the summary statistics of the original earnings variable and the new variable that we have created:

In [25]:
su earnings earnings_winsor


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
    earnings |    138,138    84136.44    252801.7   36.19316   6.36e+07
earnings_w~r |    138,138    78637.28    101010.2   2830.869   607200.1


Now we will use this new dependent variable in our regression analysis. If the outliers were not creating problems, there will be no change in the results. If they were creating problems, those problems will now be fixed. 

Let's take a look at this by first running the regression from [Module 12](econometrics/econ490-stata/12_Linear_Reg.ipynb) with the original _earnings_ variable.

In [26]:
capture drop logearnings
gen logearnings = log(earnings)
regress logearnings age 





      Source |       SS           df       MS      Number of obs   =   138,138
-------------+----------------------------------   F(1, 138136)    =    682.89
       Model |  884.189446         1  884.189446   Prob > F        =    0.0000
    Residual |  178856.105   138,136  1.29478272   R-squared       =    0.0049
-------------+----------------------------------   Adj R-squared   =    0.0049
       Total |  179740.295   138,137  1.30117416   Root MSE        =    1.1379

------------------------------------------------------------------------------
 logearnings | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0107776   .0004124    26.13   0.000     .0099692    .0115859
       _cons |   10.19145   .0188753   539.94   0.000     10.15445    10.22844
------------------------------------------------------------------------------


Now we will run this again, using the new winsorized earnings variable. 

In [27]:
capture drop logearnings_winsor
gen logearnings_winsor = log(earnings_winsor)
regress logearnings_winsor age 





      Source |       SS           df       MS      Number of obs   =   138,138
-------------+----------------------------------   F(1, 138136)    =    670.64
       Model |  825.649342         1  825.649342   Prob > F        =    0.0000
    Residual |  170063.031   138,136  1.23112752   R-squared       =    0.0048
-------------+----------------------------------   Adj R-squared   =    0.0048
       Total |  170888.681   138,137  1.23709564   Root MSE        =    1.1096

------------------------------------------------------------------------------
logearning~r | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0104147   .0004022    25.90   0.000     .0096265    .0112029
       _cons |   10.20861   .0184055   554.65   0.000     10.17253    10.24468
------------------------------------------------------------------------------


Do you think that in this case the outliers were having a significant impact before being winsorized?

#### 14.1.2 Trimming a dependent variable 

Trimming consists of replacing both values below the $a$ percentile and values above the $b$ percentile by a missing value. This is useful since any observation which equals a missing value won't be used in the regression due to Stata automatically excluding observations with missing values in the command `regress`.

Here are the commands for trimming a variable. Notice that the steps are quite similar to when we winsorized the same variable. 

In [28]:
su earnings, d

capture drop earnings_trim
gen earnings_trim = earnings
replace earnings_trim = . if earnings_trim < r(p1)
replace earnings_trim = . if earnings_trim > r(p99)



                          Earnings
-------------------------------------------------------------
      Percentiles      Smallest
 1%     2830.869       36.19316
 5%     6639.007        46.1867
10%     10220.93       51.08368       Obs             138,138
25%     20562.63       57.63494       Sum of wgt.     138,138

50%     43783.01                      Mean           84136.44
                        Largest       Std. dev.      252801.7
75%     92378.23       1.10e+07
90%     183237.5       1.38e+07       Variance       6.39e+10
95%     277388.4       3.80e+07       Skewness        143.227
99%     607200.1       6.36e+07       Kurtosis       32584.05


scalars:
                  r(N) =  138138
              r(sum_w) =  138138
               r(mean) =  84136.44398112247
                r(Var) =  63908703718.91576
                 r(sd) =  252801.7082990456
           r(skewness) =  143.2270435630384
           r(kurtosis) =  32584.05136018418
                r(sum) =  11622440098.664

And here is the result of the regression with the new dependent variable. 

In [29]:
capture drop logearnings_trim
gen logearnings_trim = log(earnings_trim)
regress logearnings_trim age 



(2,762 missing values generated)


      Source |       SS           df       MS      Number of obs   =   135,376
-------------+----------------------------------   F(1, 135374)    =    531.45
       Model |  590.410164         1  590.410164   Prob > F        =    0.0000
    Residual |  150393.096   135,374   1.1109452   R-squared       =    0.0039
-------------+----------------------------------   Adj R-squared   =    0.0039
       Total |  150983.506   135,375  1.11529829   Root MSE        =     1.054

------------------------------------------------------------------------------
logearning~m | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0089048   .0003863    23.05   0.000     .0081477    .0096618
       _cons |   10.27775   .0176777   581.40   0.000      10.2431    10.31239
------------------------------------------------------------------------------


## 14.2 Multicollinearity 

If two variables are linear combinations of one another they are multicollinear. Ultimately, Stata will not allow you to include two variables in a regression that are perfect linear combinations of one another, such as a constant, a dummy variable for male and a dummy for female (since female = 1 - male). If you try this yourself you will see that one of those variables will be dropped from the regression "because of collinearity".


In [30]:
cap drop male
gen male = sex == "M"

cap drop female 
gen female = sex == "F"

In [31]:
reg logearnings male female

note: female omitted because of collinearity.

      Source |       SS           df       MS      Number of obs   =   138,138
-------------+----------------------------------   F(1, 138136)    =   5952.64
       Model |   7425.4984         1   7425.4984   Prob > F        =    0.0000
    Residual |  172314.796   138,136   1.2474286   R-squared       =    0.0413
-------------+----------------------------------   Adj R-squared   =    0.0413
       Total |  179740.295   138,137  1.30117416   Root MSE        =    1.1169

------------------------------------------------------------------------------
 logearnings | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        male |   .5588429   .0072433    77.15   0.000     .5446463    .5730396
      female |          0  (omitted)
       _cons |   10.24279   .0063933  1602.12   0.000     10.23026    10.25532
----------------------------------------------

Is this a problem? Not really. Multicollinearity is a sign that a variable is not adding any new information. Notice that with the constant term and a male dummy we can know the mean earnings of females. In this case, the constant term is by construction the mean earnings of females, and the male dummy gives the earning premium paid to male workers.

While there are some statistical tests for multicollinearity, nothing beats having the right intuition when running a regression. If there is an obvious case where two variables contain basically the same information, you should avoid including both in the analysis. 

For instance, we might have an age variable that includes both years and months (e.g. if a baby is 1 year and 1 month old, then this age variable would be coded as 1 + 1/12 = 1.083). If we included this variable in a regression which also included an age variable that includes only years (e.g the baby's age would be coded as 1) then we would have the problem of multicollinearity. Because they are not perfectly collinear, Stata might still produce some results; however, the coefficients on these two variables would be biased.

## 14.3 Heteroskedasticity 

When we run a linear regression, we essentially split the outcome into a (linear) part explained by observables and an error term:
$$
y_i = a + b x_i + e_i
$$ 

The standard errors in our coefficients depend on $e_i^2$ (as you might remember from ECON 326). Heteroskedasticity refers to the case where the variance of this projection error depends on the observables $x_i$. For instance, the variance of wages tends to be higher for people who are university educated (some of these people have very high wages) whereas it is small for people who are non-university educated (these people tend to be concentrated in smaller paying jobs).  Stata by default assumes that the variance does not depend on the observables, which is known as homoskedasticity. It is safe to say that this is an incredibly restrictive assumption.

While there are tests for heteroskedasticity, the more empirical economists rely on including the option `robust` at the end of the `regress` command for the OLS regression to address this.

In [34]:
cap drop logearnings
gen logearnings = log(earnings)
regress logearnings age, robust





Linear regression                               Number of obs     =    138,138
                                                F(1, 138136)      =     666.21
                                                Prob > F          =     0.0000
                                                R-squared         =     0.0049
                                                Root MSE          =     1.1379

------------------------------------------------------------------------------
             |               Robust
 logearnings | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |   .0107776   .0004176    25.81   0.000     .0099592     .011596
       _cons |   10.19145   .0190919   533.81   0.000     10.15403    10.22887
------------------------------------------------------------------------------


Best practices are simply to always use robust standard errors in your ECON 490 project.

## 14.4 Non-linearity

Our regression analysis so far assumes that the relationship between our independent and explanatory variables is linear. If this is not the case, and the relationship is non-linear, then we are getting inaccurate results with our analysis. 

Let's consider an example. We know that earnings increases with age, but what if economic theory predicts that the amount that earnings increases for each year of age when workers are younger is larger than the amount that earnings increases for each year of age when workers are older? Put another way, we are asking whether earnings is increasing with age at a decreasing rate.

We could correct for this in our model by including a new interaction term that is simply age interacted with itself. You learned how to do this in [Module 13](econometrics/econ490-stata/13_Dummy.ipynb). Let's include this in the regression above, remembering that age is a continuous variable.

In [35]:
reg logearnings c.age##c.age


      Source |       SS           df       MS      Number of obs   =   138,138
-------------+----------------------------------   F(2, 138135)    =    470.65
       Model |  1216.53501         2  608.267504   Prob > F        =    0.0000
    Residual |   178523.76   138,135  1.29238614   R-squared       =    0.0068
-------------+----------------------------------   Adj R-squared   =    0.0068
       Total |  179740.295   138,137  1.30117416   Root MSE        =    1.1368

------------------------------------------------------------------------------
 logearnings | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
         age |    .079004   .0042745    18.48   0.000     .0706262    .0873819
             |
 c.age#c.age |  -.0007587   .0000473   -16.04   0.000    -.0008514    -.000666
             |
       _cons |   8.699491   .0949292    91.64   0.000     8.513431     8.88555
--------------------

There does seem to be some evidence in our regression results that this economic theory is correct, since the coefficient on the interaction term is both negative and statistically significant.

<div class="alert alert-info">


**Note:** If there is a theoretical reason for believing that non-linearity exists, Stata provides some tests for non-linearity. You can also created a scatter-plot to see if you can observe a non-linear relationship in the data. We covered that approach in [Module 9](econometrics/econ490-stata/9_Stata_Graphs.ipynb).
    
</div>

## 14.5 Wrap Up 

It is important to always follow best practices for regression analysis. Nonetheless, checking and correcting for outliers, as well as addressing heteroskedasticity, multicollinearity and non-linearity can be more of an art than a science. If you need any guidance on whether or not you need to address these issues, please be certain to speak with your instructor or TA.

## References

[How to identify and replace unusual data values](https://www.youtube.com/watch?v=jIiHb0gsyVo)